In [227]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [228]:
X = np.load('classification_problem.npy')

In [229]:
X = pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,983.990540,942.286499,661.727966,943.103271,862.790222,105.079277,-4.460563,899.619080,198.236786,372.159210,721.331665,573.659851
1,977.899475,943.009949,660.280029,953.870422,865.226379,106.982498,1.351808,903.402222,204.626221,358.535583,717.090576,515.002991
2,997.377319,943.708069,663.513123,929.276184,861.283142,113.385300,9.707563,906.708252,204.998215,349.990387,668.513977,501.894470
3,1001.554688,944.264709,650.176453,909.029968,861.296997,110.551079,-5.713619,911.538940,203.693878,349.119629,724.664612,571.145813
4,1007.083252,943.864685,662.991455,931.140503,858.884033,108.564102,-6.160424,903.476929,195.064453,367.321808,680.104126,540.926025


In [267]:
y = np.full(1000, 0)
y = np.append(y, np.full(1000, 1))
y = np.append(y, np.full(1000, 2))
y = pd.get_dummies(y, prefix='class')
y

,class_0,class_1,class_2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
2995,0,0,1
2996,0,0,1
2997,0,0,1
2998,0,0,1


In [268]:
X_train = torch.tensor(X.values)
y_train = torch.tensor(y.values, dtype=torch.float32)

In [269]:
epochs = 100
batch_size = 32
learning_rate = 0.001
n_features = len(X_train[0])
n_classes = 3

In [279]:
class Classificator(torch.nn.Module):
    def __init__(self):
        super(Classificator, self).__init__()
        self.batch_norm1 = torch.nn.BatchNorm1d(n_features)
        self.linear1 = torch.nn.Linear(n_features, 128)
        self.relu1 = torch.nn.ReLU()
        self.dropout1 = torch.nn.Dropout(0.3)
        
        self.linear2 = torch.nn.Linear(128, 64)
        self.relu2 = torch.nn.ReLU()
        self.dropout2 = torch.nn.Dropout(0.3)
        
        self.linear3 = torch.nn.Linear(64, n_classes)
        self.softmax1 = torch.nn.Softmax()
        
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.linear3(x)
        x = self.softmax1(x)
        
        return x

In [280]:
model = Classificator()

In [281]:
dataset = torch.utils.data.TensorDataset(X_train, y_train)
dataset

In [282]:
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [283]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [284]:
loss_func = torch.nn.MSELoss()

In [285]:
for epoch in (t := tqdm(range(epochs))):
    for features, target in loader:
        optimizer.zero_grad()
        pred = model(features)
        loss = loss_func(pred, target)
        act_label = target.argmax(1)
        pred_label = pred.argmax(1)
        acc = accuracy_score(pred_label, pred_label)    
        loss.backward()
        optimizer.step()
        
    t.set_description(f'epoch {epoch+1} accuracy {acc} loss {loss.item()}')
    
print('Training finished!')
print(f'accuracy = {acc} loss = {loss.item()}')

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]<ipython-input-279-5eaba1f1413d>:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax1(x)
epoch 100 accuracy 1.0 loss 0.1595524698495865: 100%|████████████████████████████████| 100/100 [00:30<00:00,  3.26it/s]

Training finished!
accuracy = 1.0 loss = 0.1595524698495865


In [ ]:
torch.save(model.state_dict(), 'model.pt')